In [3]:
"""Example client for PyTado"""
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

from PyTado.interface.interface import Tado

from home_automation.core.login import login

tado = login()


Device activation status:  DeviceActivationStatus.PENDING
Device verification URL:  https://login.tado.com/oauth2/device?user_code=LYNB2S
Starting device activation
Device activation status:  DeviceActivationStatus.COMPLETED


In [16]:
from home_automation.core.settings import settings_init

settings_init.checking_interval

10.0

In [ ]:
import sys
import os
import time
from datetime import datetime
from PyTado.interface import Tado
from pydantic import Field
from pydantic_settings import BaseSettings, SettingsConfigDict


class SettingsInit(BaseSettings):
    """Application settings loaded from the .env file or environment variables."""
    model_config = SettingsConfigDict(env_file=".env", env_file_encoding="utf-8")

    checking_interval: float = Field(..., description="Checking interval in seconds")
    error_retry_interval: float = Field(..., description="Retry interval after errors")
    min_temp: float = Field(..., description="Minimum allowed temperature")
    max_temp: float = Field(..., description="Maximum allowed temperature")
    enable_temp_limit: bool = Field(..., description="Enable temperature limit enforcement")
    save_log: bool = Field(..., description="Enable saving logs to a file")
    log_file: str = Field(..., description="Path to log file")
    max_log_lines: int = Field(..., description="Maximum number of lines before rotating the log")


settings_init = SettingsInit()


def login() -> Tado:
    """
    Perform device activation and login using PyTado.
    This typically retrieves device status and activates the device if needed.
    Returns:
        Tado: an authenticated Tado instance ready for use.
    """
    tado = Tado()
    print("Device verification URL: ", tado.device_verification_url())
    tado.device_activation()
    return tado


def count_lines(file_path: str) -> int:
    """
    Count the number of lines in a given log file.

    Args:
        file_path (str): Path to the file.

    Returns:
        int: Number of lines in the file.
    """
    with open(file_path) as f:
        return sum(1 for _ in f)



def rotate_log(log_file: str):
    """
    Rotate the log file by renaming it with a timestamp and starting a new empty log.

    Args:
        log_file (str): Path to the log file to rotate.
    """
    timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
    new_log_file = log_file.replace(".log", f"_{timestamp}.log")
    os.rename(log_file, new_log_file)
    open(log_file, "w").close()


def print_log(message: str, last_message_container: dict):
    """
    Print a timestamped log message and write it to the log file if enabled.

    Args:
        message (str): The message to log.
        last_message_container (dict): Dictionary holding the last logged message to avoid duplicates.
    """
    if message != last_message_container.get("last"):
        timestamped_message = f"{datetime.now().strftime('%d-%m-%Y %H:%M:%S')} # {message}"
        print(timestamped_message)
        if settings_init.save_log:
            try:
                with open(settings_init.log_file, "a") as log:
                    log.write(timestamped_message + "\n")
                if count_lines(settings_init.log_file) >= settings_init.max_log_lines:
                    rotate_log(settings_init.log_file)
            except Exception as e:
                print(f"Log error: {e}")
        last_message_container["last"] = message


def handle_presence(tado_instance: Tado, devices_home: list, home_state: str, last_message_container: dict):
    """
    Handle presence logic: switch between HOME and AWAY modes based on detected devices.

    Args:
        tado_instance (Tado): The Tado instance.
        devices_home (list): List of devices currently at home.
        home_state (str): The current home state ("HOME" or "AWAY").
        last_message_container (dict): Dictionary holding the last logged message.
    """
    if len(devices_home) == 0 and home_state == "HOME":
        print_log("No devices at home but state is HOME — switching to AWAY mode.", last_message_container)
        tado_instance.set_away()
    elif len(devices_home) > 0 and home_state == "AWAY":
        print_log(f"Devices {devices_home} detected at home — switching to HOME mode.", last_message_container)
        tado_instance.set_home()

    print_log("Monitoring started. Waiting for location or open window changes.", last_message_container)


def monitor_home(tado_instance: Tado, last_message_container: dict):
    """
    Monitor home presence and react to device location changes.

    Args:
        tado_instance (Tado): The Tado instance.
        last_message_container (dict): Dictionary holding the last logged message.
    """
    try:
        home_state = tado_instance.get_home_state()["presence"]
        devices_home = [
            device["name"]
            for device in tado_instance.get_mobile_devices()
            if device["settings"]["geoTrackingEnabled"]
            and device["location"] is not None
            and device["location"]["atHome"]
        ]

        handle_presence(tado_instance, devices_home, home_state, last_message_container)

    except Exception as e:
        print_log(
            f"Error checking home status: {e}. Retrying in {settings_init.error_retry_interval} seconds.",
            last_message_container,
        )
        time.sleep(settings_init.error_retry_interval)
        monitor_home(tado_instance, last_message_container)
